<h1>Estimating Interest rate parity (Gibbs Sampling)<h1\>

<h5>  To examine whether the uncovered interest rate parity (below) holds in reality<h5\>

\begin{equation}
\frac{E_t(S_{t+1}) - S_t}{S_t} = i_t - i_t^*
\end{equation}

We set up a linear regression model (below) with the change rate of the USD/KRW exchange rate as the dependent variable and the interest rate differential between Korean and US rates as the explanatory variable, using a Gibbs sampler for estimation.

\begin{equation}
\frac{S_{t+1} - S_t}{S_t} = a_1 + a_2 (i_t - i_t^*) + \epsilon_t, \quad \epsilon_t | i_t - i_t^*, \theta \sim \text{Normal}(0, \sigma^2)
\end{equation}

The notation $
\epsilon_t \mid i_t - i_t^*, \theta \sim \text{Normal}(0, \sigma^2)
$ means that the error term $\epsilon_t$, conditional on the interest rate differential $ i_t - i_t^*$ and model parameters $ \theta $, follows a normal distribution with mean zero and variance $\sigma^2 $.

<h2>Load library<h2\>

In [3]:
import numpy as np
import pandas as pd
from scipy.stats import invgamma, multivariate_normal



<h2>Generate sample data<h2\>

In [7]:
# Set random seed
np.random.seed(123)

T = 100  # Sample size

us_interest_rate = np.random.normal(1.5, 0.5, T)  # mean 1.5, sd 0.5, sample size T
korean_interest_rate = np.random.normal(2.0, 0.5, T)
exchange_rate_depreciation = korean_interest_rate - us_interest_rate + np.random.normal(0, 0.2, T)


In [8]:
# Data Preparation
Y = exchange_rate_depreciation[1:]  # depreciation rate (take out 1st sample)
X = korean_interest_rate[:-1] - us_interest_rate[:-1]  # interest rate differential (take out last sample)
X = np.column_stack((np.ones(T - 1), X))  # this adds intercept column
k = X.shape[1]                            # extracts number of columns in matrix

<h2>Setting Prior Parameters<h2\>

1. Prior distribution for $\sigma^2$:

\begin{equation}
\sigma^2 \sim \text{InverseGamma} \left( \frac{\alpha_0}{2} = \frac{50}{2}, \frac{\delta_0}{2} = \frac{1000 \times \alpha_0}{2} \right)
\end{equation}

2. Prior distribution for $\beta | \sigma^2$:

\begin{equation}
\beta | \sigma^2 \sim \text{Normal} \left( \beta_0 = \begin{bmatrix} 0 \\ 1 \end{bmatrix}, B_0 = 25 \times I_2 \right)
\end{equation}

In [ ]:
# Priors (a_0 and d_0 are parameters for the variance of residuals)
a_0 = 50         # shape parameter for inverse gamma distribution
d_0 = 50000      # prior mean is set to be approximately around 1,000 
b_0 = np.array([0, 1])  # prior mean for beta (reflecting the belief that UIRP usually holds)
B_0 = 25 * np.eye(k)  # prior variance matrix for beta

# MCMC parameters
n0 = 500    # Burn-in period. The first n0 iterations are considered the burn-in, during which the sampler "warms up" and moves closer to the true posterior distribution
n1 = 10000  # MCMC sample size

<h2>Gibbs sampling<h2\>

In [ ]:
# Gibbs Sampler
def gibbs_linear_n(Y, X, b_0, B_0, a_0, d_0, n0, n1):
    T, k = X.shape
    B_0_inv = np.linalg.inv(B_0)
    B_0_b_0 = B_0_inv @ b_0
    bm = np.zeros((n1, k))  # Store beta samples
    sig2m = np.zeros(n1)  # Store sigma^2 samples

    # Initial values
    beta = b_0
    sigma2 = 1

    for i in range(n0 + n1):
        # Update beta conditional on sigma^2
        B_n = np.linalg.inv(B_0_inv + (X.T @ X) / sigma2)
        b_n = B_n @ (B_0_b_0 + (X.T @ Y) / sigma2)
        beta = multivariate_normal.rvs(mean=b_n, cov=B_n)

        # Update sigma^2 conditional on beta
        resid = Y - X @ beta
        a_n = a_0 + T / 2
        d_n = d_0 + (resid.T @ resid) / 2
        sigma2 = invgamma.rvs(a=a_n, scale=d_n)

        if i >= n0:  # Store post burn-in samples
            bm[i - n0] = beta
            sig2m[i - n0] = sigma2

    return bm, sig2m



<h2>Running the Gibbs sampler<h2\>

In [ ]:

# bm and sig2m arrays hold the collected samples, representing 10,000 samples of beta and sigma^2. The stored samples form the MCMC sample
bm, sig2m = gibbs_linear_n(Y, X, b_0, B_0, a_0, d_0, n0, n1)

# Calculate posterior probability that the second coefficient is between 0.5 and 1.5
indicator = (bm[:, 1] > 0.5) & (bm[:, 1] < 1.5)
post_prob = np.mean(indicator)

print("Posterior probability:", post_prob)

<h2>Running the Gibbs sampler<h2\>

In [10]:
# Conclusion of Practice Exercise: The posterior means, standard errors, and 95% credible intervals for $a_1$, $a_2$, $/sigma^2$
# were estimated using Gibbs sampling. The credible intervals for include 0 and 1, respectively.
# However, due to the wide range of these intervals, it is difficult to conclude that 
# the risk-neutral equilibrium hypothesis is statistically supported.